In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/.shortcut-targets-by-id/1qNzMmGHCg5Xa63Vw3aKbZdMXvkfT2CgC/MedStar/MS_CODE/FieldNN


# Data Point

In [2]:
from recfldgrn.datapoint import PatientDP, RANGE_SIZE

PID = 'P4'
rec_folder = 'data/ProcData/RecFolder/'
Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
Pat

In [3]:
####################
RecName = 'A1C'
####################

df = Pat.get_df_rec(RecName)
df

,PID,ECID,RID,V,DT
0,P4,EC196,3500,23.831523,2023-03-31
1,P4,EC36,5382,2.313767,2023-04-13
2,P4,EC197,3499,47.156259,2023-04-14


# Task Selection

In [4]:
TaskName = 'Task2YearXXX'

ELIG_PAT_SET = [f'P{i}' for i in range(0, 8)]
ELIG_PAT_SET

['P0', 'P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7']

In [5]:
import os
import pandas as pd

Tensor_folder = 'data/ProcData/TensorFolder'
TaskTensor_folder = os.path.join(Tensor_folder, TaskName)

if not os.path.exists(TaskTensor_folder): os.makedirs(TaskTensor_folder)

print(TaskTensor_folder)

data/ProcData/TensorFolder/Task2YearXXX


In [6]:
######################### Change this
RecName = 'A1C'
######################### 


df = pd.DataFrame()
for PID in ELIG_PAT_SET:
    Pat = PatientDP(PID, rec_folder, RANGE_SIZE)
    dfx = Pat.get_df_rec(RecName)
    
    ###################################
    # TODO: make some operation to select good dfx
    dfx = dfx 
    ###################################
    
    df = pd.concat([df, dfx])
    
print(df.shape)
df

(27, 5)


,PID,ECID,RID,V,DT
0,P0,EC13,5050,36.499557,2023-03-27
1,P0,EC14,4651,36.247254,2023-04-06
2,P0,EC18,5801,9.260195,2023-04-08
3,P0,EC87,3542,44.569271,2023-04-09
4,P0,EC201,5800,17.660021,2023-04-12
0,P1,EC33,5799,15.500044,2023-03-25
1,P1,EC0,4649,45.309869,2023-04-02
2,P1,EC26,6204,21.664842,2023-04-06
3,P1,EC2,3541,31.810007,2023-04-06
4,P1,EC28,3897,32.831454,2023-04-08


# Load Meta Info

In [7]:
import os
import pandas as pd

fldgrn_folder = 'data/ProcData/FldGrnInfo'

fullfldgrn_file = os.path.join(fldgrn_folder, RecName + '.p')

df_FieldGrainInfo = pd.read_pickle(fullfldgrn_file)
df_FieldGrainInfo

,RecName,prefix_ids,focal_ids,field_list,field,grain,field2grain,recfield2grain,Vocab
0,A1C,"[PID, ECID]",[RID],"[V, DT]",V,A1CNumeDftGrn,V-A1CNumeDftGrn,A1C@V-A1CNumeDftGrn,"{'v2idx': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,..."
1,A1C,"[PID, ECID]",[RID],"[V, DT]",DT,DTDftGrn,DT-DTDftGrn,A1C@DT-DTDftGrn,"{'idx2v': {0: '_missing', 1: 'Yx', 2: 'Y2010',..."


# RecFldGrn `A1C@V-A1CNumeDftGrn`

In [8]:
from recfldgrn.grains import Field2Grain_2_get_GrnStrList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnIdxList_Fn
from recfldgrn.grains import Field2Grain_2_get_GrnWgtList_Fn
from recfldgrn.utils import generate_grain_vocab_info
from recfldgrn.utils import get_compressed_df
from recfldgrn.datapoint import write_df_to_folders

In [9]:
from recfldgrn.utils import get_highorder_input_idx

In [10]:
TaskTensor_folder

'data/ProcData/TensorFolder/Task2YearXXX'

In [11]:
######################### Change this
recfield2grain = 'A1C@V-A1CNumeDftGrn'
#########################


d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)



fullTaskTensor_folder = os.path.join(TaskTensor_folder, recfield2grain)
print(df.shape, fullTaskTensor_folder)
if not os.path.exists(fullTaskTensor_folder): os.makedirs(fullTaskTensor_folder)
write_df_to_folders(recfield2grain, fullTaskTensor_folder, df_p)

df_p

(27, 6) data/ProcData/TensorFolder/Task2YearXXX/A1C@V-A1CNumeDftGrn


,PID,P-EC-A1C@V-A1CNumeDftGrn_wgt
0,P0,"[[[0, 0, 0, 0, 8.571301925765386, 1, 1.0, 1.0,..."
1,P1,"[[[0, 0, 0, 0, 11.08853405587856, 1, 1.0, 1.0,..."
2,P2,"[[[0, 0, 0, 0, 0.38586582765888877, 1, 1.0, 1...."
3,P3,"[[[0, 0, 0, 0, 7.022274046376522, 1, 1.0, 1.0,..."
4,P4,"[[[0, 0, 0, 0, 4.951863775201697, 1, 1.0, 1.0,..."
5,P5,"[[[0, 0, 0, 0, 8.842130499673823, 1, 1.0, 1.0,..."
6,P6,"[[[0, 0, 0, 0.6511763706255762, 0, 0, 0.0, 0.0..."
7,P7,"[[[0, 0, 0, 0, 3.0705198239387923, 1, 1.0, 1.0..."


# RecFldGrn `A1C@DT-DTDftGrn`

In [12]:
######################### Change this
recfield2grain = 'A1C@DT-DTDftGrn'
#########################

d = df_FieldGrainInfo[df_FieldGrainInfo['recfield2grain'] == recfield2grain].iloc[0]
v2idx = d['Vocab']['v2idx']
prefix_ids = d['prefix_ids']
focal_ids = d['focal_ids']

df_p = get_highorder_input_idx(df, recfield2grain, 
                               v2idx, prefix_ids, focal_ids,
                               Field2Grain_2_get_GrnIdxList_Fn, 
                               Field2Grain_2_get_GrnWgtList_Fn)


fullTaskTensor_folder = os.path.join(TaskTensor_folder, recfield2grain)
print(df.shape, fullTaskTensor_folder)
if not os.path.exists(fullTaskTensor_folder): os.makedirs(fullTaskTensor_folder)
write_df_to_folders(recfield2grain, fullTaskTensor_folder, df_p)

df_p

(27, 7) data/ProcData/TensorFolder/Task2YearXXX/A1C@DT-DTDftGrn


,PID,P-EC-A1C@DT-DTDftGrn_idx
0,P0,"[[[15, 25, 62, 68, 73, 94, 137]], [[15, 26, 41..."
1,P1,"[[[15, 26, 37, 69, 79, 94, 137]], [[15, 26, 41..."
2,P2,"[[[15, 26, 46, 69, 74, 96, 137]], [[15, 25, 60..."
3,P3,"[[[15, 26, 50, 69, 78, 96, 137]]]"
4,P4,"[[[15, 25, 66, 68, 77, 94, 137]], [[15, 26, 49..."
5,P5,"[[[15, 26, 39, 69, 74, 95, 137]], [[15, 26, 46..."
6,P6,"[[[15, 26, 47, 69, 75, 96, 137]], [[15, 26, 44..."
7,P7,"[[[15, 26, 41, 69, 76, 95, 137]]]"
